# PDF et Web Search : Sources Documentaires avec OpenAI

Ce notebook explore deux fonctionnalités puissantes de l'API OpenAI :
- **Support PDF direct** : Envoyer des documents PDF aux modèles vision
- **Web Search** : Accéder à des informations en temps réel

**Objectifs :**
- Charger et analyser des PDFs via l'API
- Utiliser l'outil web_search pour la recherche en temps réel
- Combiner documents et recherche pour des réponses enrichies

**Prérequis :** Notebook 1 (OpenAI Intro)

**Durée estimée :** 50 minutes

In [ ]:
# Installation des dépendances
%pip install openai python-dotenv reportlab pillow -q

import os
import base64
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv('../.env')
client = OpenAI()

# Charger le modèle depuis .env ou utiliser gpt-4o par défaut
DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")
BATCH_MODE = os.getenv("BATCH_MODE", "false").lower() == "true"

print("Client OpenAI initialisé !")
print(f"Modèle par défaut: {DEFAULT_MODEL}")
print(f"Mode: {'Batch' if BATCH_MODE else 'Interactive'}")

## 1. Support de Documents dans l'API OpenAI

Les modèles OpenAI avec capacités vision peuvent traiter des images et, via l'Assistants API, des fichiers PDF :

**Modèles compatibles :**
- `gpt-4o` et `gpt-4o-mini` (avec vision)
- Tous les modèles vision de la famille GPT-4

**Approches pour les documents :**

| Approche | API | Avantages | Limites |
|----------|-----|-----------|---------|
| **Image directe** | Chat Completions | Simple, rapide | Uniquement images (png, jpg, gif, webp) |
| **Assistants + Files** | Assistants API | Supporte PDF natif | Plus complexe, coût stockage |
| **Conversion image** | Chat Completions | Universel | Perte de qualité potentielle |

**Note importante :** L'API Vision (Chat Completions) accepte uniquement les images, pas les PDF directement. Pour les PDF, utilisez l'Assistants API ou convertissez en images.

In [ ]:
# Créer un document de test sous forme d'IMAGE (pour l'API Vision)
from PIL import Image, ImageDraw, ImageFont
import io

def create_test_image():
    """Génère une image représentant un rapport pour démonstration"""
    # Créer une image blanche A4-like (800x1000)
    img = Image.new('RGB', (800, 1000), color='white')
    draw = ImageDraw.Draw(img)
    
    # Utiliser une police par défaut
    try:
        font_title = ImageFont.truetype("arial.ttf", 36)
        font_normal = ImageFont.truetype("arial.ttf", 18)
        font_small = ImageFont.truetype("arial.ttf", 14)
    except:
        # Fallback si police non disponible
        font_title = ImageFont.load_default()
        font_normal = ImageFont.load_default()
        font_small = ImageFont.load_default()
    
    # Dessiner le contenu
    y = 50
    draw.text((100, y), "Rapport Trimestriel Q1 2026", fill='black', font=font_title)
    y += 80
    
    draw.text((100, y), "Résumé Exécutif", fill='darkblue', font=font_normal)
    y += 40
    
    lines = [
        "- Chiffre d'affaires: 2.5M EUR (+15%)",
        "- Nouveaux clients: 150 (+25%)",
        "- Satisfaction client: 4.5/5",
        "",
        "Points clés:",
        "1. Lancement réussi du produit Alpha",
        "2. Expansion sur le marché européen",
        "3. Recrutement de 20 ingénieurs",
        "",
        "Perspectives Q2: Objectif 3M EUR (+20%)"
    ]
    
    for line in lines:
        draw.text((100, y), line, fill='black', font=font_normal)
        y += 30
    
    # Sauvegarder en bytes
    buffer = io.BytesIO()
    img.save(buffer, format='PNG')
    buffer.seek(0)
    return buffer.getvalue()

# Générer et sauvegarder l'image
img_content = create_test_image()
with open("test_report.png", "wb") as f:
    f.write(img_content)

print("✓ Image de rapport créée: test_report.png")
print(f"  Taille: {len(img_content)} octets")

In [ ]:
# Analyser l'image via l'API OpenAI Vision
# Charger et encoder en base64
with open("test_report.png", "rb") as f:
    img_base64 = base64.b64encode(f.read()).decode()

print(f"Envoi de l'image au modèle {DEFAULT_MODEL}...\n")

# Envoyer au modèle avec vision
response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{
        "role": "user",
        "content": [
            {
                "type": "text", 
                "text": "Analyse ce rapport et donne-moi les 3 points clés avec les chiffres associés."
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{img_base64}"
                }
            }
        ]
    }],
    max_tokens=500
)

print("=== Analyse du document ===")
print(response.choices[0].message.content)
print(f"\nTokens utilisés: {response.usage.total_tokens}")

## 2. Web Search avec l'API OpenAI

L'outil **web_search_preview** permet d'effectuer des recherches en temps réel et d'enrichir les réponses avec des informations actualisées.

**Caractéristiques :**
- Accès à des informations en temps réel (actualités, cours boursiers, météo, etc.)
- **Citations automatiques** : Le modèle cite ses sources
- Disponible via la **Responses API** (bêta)
- Modèles compatibles : `gpt-4o`, `gpt-4o-mini`

**Différence avec Chat Completions :**
- Responses API : Interface simplifiée avec `input` et `output`
- Support natif des outils comme web_search
- Moins de contrôle sur les paramètres avancés

**Note importante :** Cette fonctionnalité est en préversion et peut évoluer.

In [ ]:
# Web Search basique via Responses API
print("Recherche web en cours...\n")

response = client.responses.create(
    model=DEFAULT_MODEL,
    tools=[{"type": "web_search_preview"}],
    input="Quelles sont les dernières avancées majeures en intelligence artificielle en janvier 2026?"
)

print("=== Recherche Web : IA en 2026 ===")
for item in response.output:
    if hasattr(item, 'content'):
        print(item.content)
        print()

In [ ]:
# Web Search avec données financières en temps réel
print("Recherche d'informations financières...\n")

response = client.responses.create(
    model=DEFAULT_MODEL,
    tools=[{"type": "web_search_preview"}],
    input="Quel est le cours actuel de l'action Apple (AAPL) et quelles sont ses performances sur les 3 derniers mois?"
)

print("=== Informations financières en temps réel ===")
for item in response.output:
    if hasattr(item, 'content'):
        print(item.content)
        print()

# Note: Les citations sont incluses automatiquement dans la réponse

## 3. Combiner PDF et Web Search

Le véritable pouvoir vient de la **combinaison** de ces deux fonctionnalités :

**Cas d'usage :**
- **Analyse de rapports enrichie** : Comparer les données d'un rapport PDF avec les tendances actuelles
- **Fact-checking** : Vérifier des affirmations dans un document avec des sources web
- **Veille concurrentielle** : Analyser un rapport interne et le contextualiser avec l'actualité du secteur
- **Actualisation de documents** : Identifier les informations obsolètes dans un PDF

**Workflow typique :**
1. Extraire les informations clés du PDF
2. Formuler une requête web basée sur ces informations
3. Combiner les deux sources pour une analyse enrichie

In [ ]:
# Workflow combiné : Document Image + Web Search
print("=== ÉTAPE 1 : Extraction des informations du document ===\n")

# Analyser le document image pour extraire les données financières
doc_analysis = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{
        "role": "user",
        "content": [
            {
                "type": "text", 
                "text": "Extrais le chiffre d'affaires, le taux de croissance et les perspectives Q2 de ce rapport."
            },
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/png;base64,{img_base64}"}
            }
        ]
    }],
    max_tokens=300
)

ca_info = doc_analysis.choices[0].message.content
print("Données extraites du document:")
print(ca_info)
print()

# Construire une requête web contextualisée
print("=== ÉTAPE 2 : Enrichissement avec données web ===\n")

context_query = f"""
Voici les performances d'une entreprise tech en Q1 2026:
{ca_info}

Compare ces résultats avec:
1. Les performances moyennes du secteur tech en 2026
2. Les tendances de croissance actuelles
3. Les perspectives pour Q2 2026

Fournis une analyse comparative brève.
"""

web_context = client.responses.create(
    model=DEFAULT_MODEL,
    tools=[{"type": "web_search_preview"}],
    input=context_query
)

print("=== Analyse enrichie (Document + Web) ===")
for item in web_context.output:
    if hasattr(item, 'content'):
        print(item.content)
        print()

## 4. Exemple avancé : Vérification de faits

Un autre cas d'usage puissant : vérifier les affirmations d'un document PDF avec des sources web récentes.

In [ ]:
# Fact-checking : Vérifier une affirmation du document
print("=== Vérification de faits ===\n")

# Extraire une affirmation spécifique
claim_extraction = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Quelle est l'affirmation principale sur les perspectives de croissance dans ce rapport?"
            },
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/png;base64,{img_base64}"}
            }
        ]
    }],
    max_tokens=200
)

claim = claim_extraction.choices[0].message.content
print(f"Affirmation extraite: {claim}\n")

# Vérifier avec web search
verification_query = f"""
Affirmation à vérifier: {claim}

Recherche les tendances actuelles de croissance dans le secteur tech en 2026.
Cette affirmation est-elle réaliste? Cite des sources récentes.
"""

verification = client.responses.create(
    model=DEFAULT_MODEL,
    tools=[{"type": "web_search_preview"}],
    input=verification_query
)

print("=== Résultat de la vérification ===")
for item in verification.output:
    if hasattr(item, 'content'):
        print(item.content)

## 5. Limitations et bonnes pratiques

### Limitations PDF

| Contrainte | Limite | Impact |
|------------|--------|--------|
| **Pages** | 100 max | Documents longs nécessitent découpage |
| **Taille** | 32 MB | PDFs avec images haute résolution peuvent dépasser |
| **Coût** | 1 page = 1 image | Un PDF de 10 pages coûte autant que 10 images |
| **Qualité OCR** | Variable | Texte dans images peut être mal reconnu |

### Bonnes pratiques PDF

1. **Optimiser les PDF** : Compresser avant envoi
2. **Découper si nécessaire** : Traiter par sections pour documents longs
3. **Privilégier le texte** : PDFs textuels > PDFs scannés
4. **Vérifier les coûts** : Calculer tokens avant traitement massif

### Limitations Web Search

- **Latence** : Requêtes web ajoutent 2-5 secondes
- **Fiabilité** : Sources web peuvent être incorrectes
- **Coût** : Requêtes web consomment plus de tokens
- **Disponibilité** : Fonctionnalité en préversion (bêta)

### Bonnes pratiques Web Search

1. **Vérifier les citations** : Toujours consulter les sources mentionnées
2. **Queries spécifiques** : Plus la requête est précise, meilleurs les résultats
3. **Combiner avec connaissances** : Ne pas tout déléguer au web search
4. **Gérer les erreurs** : Prévoir des fallbacks si la recherche échoue

### Estimation des coûts

**Exemple de calcul pour GPT-4o-mini :**
- PDF 10 pages : ~10 images × 2833 tokens = ~28,000 tokens input
- Web search : ~500-1000 tokens supplémentaires
- Total pour notre workflow : ~30,000 tokens input + 500 output
- Coût estimé : $0.045 (tarif janvier 2026)

**Recommandation :** Toujours tester avec `gpt-4o-mini` avant d'utiliser `gpt-4o` (10× plus cher).

In [ ]:
# Nettoyage : Supprimer l'image de test
import os

if os.path.exists("test_report.png"):
    os.remove("test_report.png")
    print("✓ Fichier de test supprimé")
else:
    print("Aucun fichier à nettoyer")

## Conclusion

### Ce que nous avons appris

1. **Support PDF natif** : Les modèles vision peuvent analyser des PDFs directement
   - Encodage base64 pour envoi direct
   - Limites : 100 pages, 32 MB
   - Coût : 1 page = 1 image

2. **Web Search** : Accès en temps réel à l'information
   - Via Responses API avec `web_search_preview`
   - Citations automatiques
   - Idéal pour données actuelles

3. **Combinaison PDF + Web** : Analyses enrichies
   - Extraction de données PDF
   - Contextualisation avec sources web
   - Fact-checking et vérification

### Cas d'usage professionnels

| Domaine | Application |
|---------|-------------|
| **Finance** | Analyse de rapports avec données marché en temps réel |
| **Juridique** | Vérification de conformité avec réglementations actuelles |
| **Recherche** | Actualisation de revues de littérature |
| **Consulting** | Benchmarking clients vs. tendances secteur |
| **Journalism** | Fact-checking automatisé de documents |

### Exercices suggérés

1. **Niveau débutant** :
   - Analyser votre CV (PDF) et obtenir des conseils basés sur les tendances emploi actuelles
   - Créer un résumé enrichi d'un article de recherche

2. **Niveau intermédiaire** :
   - Développer un système de veille qui compare des rapports trimestriels successifs avec l'actualité
   - Créer un fact-checker pour articles de presse (PDF) vs. sources web

3. **Niveau avancé** :
   - Pipeline automatisé d'analyse de documents contractuels avec vérification de conformité légale
   - Système de recommandation qui analyse des rapports internes et suggère des actions basées sur les tendances marché

### Prochaines étapes

- **Notebook 7** : Structured Outputs (JSON Schema forcé)
- **Notebook 8** : Function Calling avancé
- **Notebook 9** : Assistants API et Code Interpreter

### Ressources complémentaires

- [Documentation OpenAI - Vision](https://platform.openai.com/docs/guides/vision)
- [Responses API Reference](https://platform.openai.com/docs/api-reference/responses)
- [Guide des prix OpenAI](https://openai.com/api/pricing/)